## Лабораторная работа №1. Мета-признаки

In [7]:
import numpy as np
import pandas as pd

all_data = pd.read_csv('OpenML/data.csv')
list_with_class_problems = list()
for index, target in enumerate(all_data['target']):
    if target == 'class' or target == 'binaryClass' or target == 'Class':
        list_with_class_problems.append((all_data['id'][index], target))

### Считываем данные

In [8]:
from scipy.io import arff

data = list()
for dt in list_with_class_problems:
    fileName = str(dt[0]) + '.arff'
    try:
        data.append((pd.DataFrame(arff.loadarff(open('OpenML/data/' + fileName))[0]), dt[1]))
    except:
        continue

print("Available sata size : " + str(len(data)))

Available sata size : 356


### Считаем базовые мета-признаки

In [9]:
from pandas.api.types import is_numeric_dtype


def compare_lists(list1, list2):
    if len(list1) != len(list2):
        return False
    for i in range(len(list1)):
        if list2[i] != list1[i]:
            return False
    return True


def get_numeric_features_count(df, target):
    columns_count = len(df.iloc[0])
    count = 0
    for index in range(columns_count):
        if is_numeric_dtype(df.iloc[:, index]) and not compare_lists(df.iloc[:, index], df.loc[:, target]):
            count += 1
    return count


def get_features_count(df):
    return len(df.iloc[0]) - 1


def get_classes_count(df, target):
    return len(set(df.loc[:, target]))


def get_objects_count(df):
    return len(df.iloc[:, -1])


base_meta_features = {}
for (index, (d, target)) in enumerate(data):
    base_meta_features[index] = [
        get_objects_count(d),
        get_classes_count(d, target),
        get_features_count(d),
        get_features_count(d) - get_numeric_features_count(d, target)
    ]

### Считаем статистические мета-признаки для цифр

In [10]:
import scipy.stats as sps


def get_numeric_features(df, target):
    columns_count = len(df.iloc[0])
    answer = list()
    for index in range(columns_count):
        if is_numeric_dtype(df.iloc[:, index]) and not compare_lists(df.iloc[:, index], df.loc[:, target]):
            answer.append(df.iloc[:, index])
    return answer


def do_numeric_func(data, func_name):
    try:
        if func_name == 'min':
            return np.min(data)
        elif func_name == 'max':
            return np.max(data)
        elif func_name == 'avg':
            return np.average(data)
        elif func_name == 'var':
            return np.var(data)
        elif func_name == 'median':
            return np.median(data)
        elif func_name == 'skew':
            return sps.skew(data)
        elif func_name == 'kurtosis':
            return sps.kurtosis(data)
    except:
        return 0


numeric_functions = {'min', 'max', 'avg', 'var', 'median', 'skew', 'kurtosis'}

numeric_meta_features = {}
for (index, (d, target)) in enumerate(data):
    numeric_features = get_numeric_features(d, target)
    answer = list()
    for f1 in numeric_functions:
        for f2 in numeric_functions:
            apply_func = list()
            for col in numeric_features:
                apply_func.append(do_numeric_func(col, f1))
            answer.append(do_numeric_func(apply_func, f2))
    numeric_meta_features[index] = answer

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:28: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:26: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/Users/egor.bazhenov/PycharmProjects/autoML/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/egor.bazhenov/PycharmProjects/autoML/venv/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning:

invalid value encountered in divide

/Users/egor.bazhenov/PycharmProjects/autoML/venv/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning:

invalid value encountered in double_scalars

/

### Считаем статистические мета-признаки для категорий

In [11]:
def get_category_features(df, target):
    columns_count = len(df.iloc[0])
    answer = list()
    for index in range(columns_count):
        if not is_numeric_dtype(df.iloc[:, index]) and not compare_lists(df.iloc[:, index], df.loc[:, target]):
            answer.append(df.iloc[:, index])
    return answer


def category_to_probability(df):
    op = {}
    ans = list()
    for key in df:
        if op.keys().__contains__(key):
            op[key] = op[key] + 1
        else:
            op[key] = 1
    for (key, value) in op:
        ans.append(value / len(df))
    return ans


def do_category_func(data, func_name):
    try:
        if func_name == 'values':
            return len(set(data))
        elif func_name == 'entropy':
            return sps.entropy(category_to_probability(data))
    except:
        return 0


category_functions = {'entropy', 'values'}

category_meta_features = {}
for (index, (d, target)) in enumerate(data):
    numeric_features = get_category_features(d, target)
    answer = list()
    for f1 in numeric_functions:
        for f2 in category_functions:
            apply_func = list()
            for col in numeric_features:
                apply_func.append(do_category_func(col, f2))
            answer.append(do_numeric_func(apply_func, f1))
    category_meta_features[index] = answer

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:28: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:26: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.



### Считаем структурные мета-признаки

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import tree


def drop_category_features(df, target):
    columns_count = len(df.iloc[0])
    answer = list()
    for index in range(columns_count):
        if is_numeric_dtype(df.iloc[:, index]) and not compare_lists(df.iloc[:, index], df.loc[:, target]):
            answer.append(df.iloc[:, index])
    return pd.DataFrame(np.transpose(answer))


lab = preprocessing.LabelEncoder()
structure_meta_features = {}
for (index, (d, target)) in enumerate(data):
    cur_d = d.copy()
    numeric_features = get_category_features(cur_d, target)
    labels = lab.fit_transform(d.loc[:, target])
    cur_d = drop_category_features(cur_d, target)
    li = list()
    li.append(cur_d)
    for r in numeric_features:
        li.append(pd.get_dummies(r))
    cur_d = pd.concat(li, axis=1).fillna(0)

    neigh = KNeighborsClassifier(n_neighbors=len(set(labels)))
    neigh = neigh.fit(cur_d, labels)

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(cur_d, labels)

    list_n = list()
    list_c = list()
    for (i, ob) in cur_d.iterrows():
        list_c.append(clf.decision_path([ob]).getnnz())
        list_n.append(np.min(neigh.kneighbors([ob])))

    answer = list()
    for f in numeric_functions:
        answer.append(do_numeric_func(list_n, f))
        answer.append(do_numeric_func(list_c, f))

    structure_meta_features[index] = answer

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:28: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:26: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:28: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:26: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Resu

### Выводим готовый набор мета-признаков

In [13]:
new_data = list()
for k in range(len(data)):
    cur_f = list(base_meta_features[k]) + numeric_meta_features[k] + category_meta_features[k] + structure_meta_features[k]
    new_data.append(cur_f)
print(pd.DataFrame(new_data).fillna(0))

      0   1   2   3            4             5         6             7   \
0    148   4  18  15         8.00  4.666667e+00 -1.500000  5.000000e+00   
1     76   2  45  45         0.00  0.000000e+00  0.000000  0.000000e+00   
2    339   2  17  17         0.00  0.000000e+00  0.000000  0.000000e+00   
3    214   2   9   0        75.41  5.003625e+02  3.250157  1.426377e+01   
4    148   2  18  15         8.00  4.666667e+00 -1.500000  5.000000e+00   
..   ...  ..  ..  ..          ...           ...       ...           ...   
351  846   2  18   0      1018.00  4.884154e+04  7.584331  1.988889e+02   
352  214   2   9   0        75.41  5.003625e+02  3.250157  1.426377e+01   
353  625   2   4   0         5.00  0.000000e+00  0.000000  5.000000e+00   
354   57   2  11   7  17953372.00  6.007687e+13 -0.666686  4.528421e+06   
355  226   2  69  69         0.00  0.000000e+00  0.000000  0.000000e+00   

           8         9   ...   71         72   73        74   75   76   77  \
0        4.00      3.

### Проверяем, что набор мета-признаков не зависит от порядка строк и столбцов

In [14]:
new_base_meta_features = {}
new_numeric_meta_features = {}
new_category_meta_features = {}
new_structure_meta_features = {}
for (index, (d, target)) in enumerate(data):
    d = d.sample(frac=1).sample(frac=1,axis=1)
    new_base_meta_features[index] = [
        get_objects_count(d),
        get_classes_count(d, target),
        get_features_count(d),
        get_features_count(d) - get_numeric_features_count(d, target)
    ]

    numeric_features = get_numeric_features(d, target)
    answer = list()
    for f1 in numeric_functions:
        for f2 in numeric_functions:
            apply_func = list()
            for col in numeric_features:
                apply_func.append(do_numeric_func(col, f1))
            answer.append(do_numeric_func(apply_func, f2))
    new_numeric_meta_features[index] = answer

    category_features = get_category_features(d, target)
    answer = list()
    for f1 in numeric_functions:
        for f2 in category_functions:
            apply_func = list()
            for col in category_features:
                apply_func.append(do_category_func(col, f2))
            answer.append(do_numeric_func(apply_func, f1))
    new_category_meta_features[index] = answer

    cur_d = d.copy()
    numeric_features = get_category_features(cur_d, target)
    labels = lab.fit_transform(d.loc[:, target])
    cur_d = drop_category_features(cur_d, target)
    li = list()
    li.append(cur_d)
    for r in numeric_features:
        li.append(pd.get_dummies(r))
    cur_d = pd.concat(li, axis=1).fillna(0)

    neigh = KNeighborsClassifier(n_neighbors=len(set(labels)))
    neigh = neigh.fit(cur_d, labels)

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(cur_d, labels)

    list_n = list()
    list_c = list()
    for (i, ob) in cur_d.iterrows():
        list_c.append(clf.decision_path([ob]).getnnz())
        list_n.append(np.min(neigh.kneighbors([ob])))

    answer = list()
    for f in numeric_functions:
        answer.append(do_numeric_func(list_n, f))
        answer.append(do_numeric_func(list_c, f))

    new_structure_meta_features[index] = answer
    break

print(pd.DataFrame(list(
    new_base_meta_features[0] + new_numeric_meta_features[0] + new_category_meta_features[0] + new_structure_meta_features[0])).fillna(
    0).transpose())

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:28: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/var/folders/q0/lmw4x3p12rx66d0_w3_0fdz00000gp/T/ipykernel_26418/1940479698.py:26: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.



      0    1     2     3    4         5    6    7    8    9   ...   71  \
0  148.0  4.0  18.0  15.0  8.0  4.666667 -1.5  5.0  4.0  3.0  ...  0.0   

         72   73        74   75   76   77   78   79        80  
0  0.365736  0.0  6.959459  0.0  7.0  0.0  4.0  0.0  0.671184  

[1 rows x 81 columns]


### Определяем наилучший алгоритм для каждого набора данных

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm


def accur_by_model(X, y, model):
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        new_model = model.fit(X_train, y_train)
        pred_y = new_model.predict(X_test)
        return accuracy_score(y_test, pred_y)
    except:
        return 0


targets = list()
for (index, (d, target)) in enumerate(data):
    cur_d = d.copy()
    numeric_features = get_category_features(cur_d, target)
    labels = lab.fit_transform(d.loc[:, target])
    cur_d = drop_category_features(cur_d, target)
    li = list()
    li.append(cur_d)
    for r in numeric_features:
        li.append(pd.get_dummies(r))
    cur_d = pd.concat(li, axis=1).fillna(0)

    algos = [KNeighborsClassifier(n_neighbors=len(set(labels))), tree.DecisionTreeClassifier(), svm.SVC()]
    acs = list()
    for algo in algos:
        acs.append(accur_by_model(cur_d, labels, algo))
    if acs[0] >= acs[1] and acs[0] >= acs[2]:
        targets.append("KNN")
    elif acs[1] >= acs[0] and acs[1] >= acs[2]:
        targets.append("DT")
    else:
        targets.append("SVM")

### Создаем новый набор данных

In [16]:
pd.concat([pd.DataFrame(new_data).fillna(0), pd.DataFrame(targets)], axis=1).to_csv('new_data.csv')

### Визуализация нового набора данных

In [17]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

df = pd.read_csv('new_data.csv')
X = list()
for col in df:
    X.append(df[col])
X.pop()
X = pd.DataFrame(X).transpose()

pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=2))])
X = pipeline.fit_transform(X)
Y = df.iloc[:, -1]
new_X = list()
new_Y = list()
for (index, (x, y)) in enumerate(X):
    if -5 <= x <= 5 and -5 <= y <= 5:
        new_X.append([x, y])
        new_Y.append(Y[index])

fig = px.scatter(new_X, x=0, y=1, color=new_Y)
fig.show()

### Тестируем некоторые алгоритмы обучения с учителем на нашем наборе данных

In [12]:
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn import svm
import pandas as pd

def cross_by_model(X, y, model):
    try:
        score = cross_val_score(model, X, y, cv=7)
        return score.mean()
    except:
        return 0

df = pd.read_csv('new_data.csv')
X = list()
for col in df:
    X.append(df[col])
X.pop()
X = pd.DataFrame(X).transpose()

y = df.iloc[:, -1]
pp = {}
for yy in y:
    if not pp.keys().__contains__(yy):
        pp[yy] = 1
    else:
        pp[yy] = pp[yy] + 1

for yy in pp.keys():
    print(pp[yy] / len(y))

algos = [svm.SVC(kernel='linear', gamma=1), CatBoostClassifier(iterations=500, depth=6, learning_rate=0.01,
                                                             metric_period=500, task_type='CPU')]
results = list()
for algo in algos:
    results.append(cross_by_model(X, y, algo))

print(results)

0.24719101123595505
0.4606741573033708
0.29213483146067415
0:	learn: 1.0966088	total: 30.3ms	remaining: 15.1s


/Users/egor.bazhenov/PycharmProjects/autoML/venv/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


499:	learn: 0.6459983	total: 6.65s	remaining: 0us
0:	learn: 1.0959536	total: 24.1ms	remaining: 12s
499:	learn: 0.6320214	total: 5.63s	remaining: 0us
0:	learn: 1.0964693	total: 22.2ms	remaining: 11.1s
499:	learn: 0.6502947	total: 5.57s	remaining: 0us
0:	learn: 1.0963877	total: 19.2ms	remaining: 9.59s
499:	learn: 0.6546806	total: 4.7s	remaining: 0us
0:	learn: 1.0960945	total: 16ms	remaining: 7.98s
499:	learn: 0.6572685	total: 4.47s	remaining: 0us
0:	learn: 1.0963975	total: 18.1ms	remaining: 9.02s
499:	learn: 0.6346857	total: 4.32s	remaining: 0us
0:	learn: 1.0967677	total: 19.1ms	remaining: 9.55s
499:	learn: 0.6591791	total: 4.8s	remaining: 0us
[0.5607282913165266, 0.6114285714285714]
